In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, Dropout, Lambda
from keras import backend as K
from sklearn.metrics import roc_auc_score
import random
import warnings


warnings.filterwarnings("ignore")
random.seed(3)
np.random.seed(3)

batch_size = 1024
seed = 3
tf.set_random_seed(seed)

# 自定义AUC指标，使用tf.py_func包装scikit-learn的roc_auc_score
def auc_roc(y_true, y_pred):
    def fallback_auc(y_true, y_pred):
        try:
            return roc_auc_score(y_true, y_pred)
        except:
            return 0.5
    return tf.py_func(fallback_auc, (y_true, y_pred), tf.float64)

def to_categorical(y, num_classes=None, dtype='float32'):
    y = np.array(y, dtype='int')
    input_shape = y.shape
    if input_shape and input_shape[-1] == 1 and len(input_shape) > 1:
        input_shape = tuple(input_shape[:-1])
    y = y.ravel()
    if not num_classes:
        num_classes = np.max(y) + 1
    n = y.shape[0]
    categorical = np.zeros((n, num_classes), dtype=dtype)
    categorical[np.arange(n), y] = 1
    output_shape = input_shape + (num_classes,)
    categorical = np.reshape(categorical, output_shape)
    return categorical

def data_preparation():
    column_names = ['age', 'class_worker', 'det_ind_code', 'det_occ_code', 'education', 'wage_per_hour', 'hs_college',
                    'marital_stat', 'major_ind_code', 'major_occ_code', 'race', 'hisp_origin', 'sex', 'union_member',
                    'unemp_reason', 'full_or_part_emp', 'capital_gains', 'capital_losses', 'stock_dividends',
                    'tax_filer_stat', 'region_prev_res', 'state_prev_res', 'det_hh_fam_stat', 'det_hh_summ',
                    'instance_weight', 'mig_chg_msa', 'mig_chg_reg', 'mig_move_reg', 'mig_same', 'mig_prev_sunbelt',
                    'num_emp', 'fam_under_18', 'country_father', 'country_mother', 'country_self', 'citizenship',
                    'own_or_self', 'vet_question', 'vet_benefits', 'weeks_worked', 'year', 'income_50k']

    train_df = pd.read_csv('./DATA/census-income.data', delimiter=',', header=None, index_col=None, names=column_names)
    test_df = pd.read_csv('./DATA/census-income.test', delimiter=',', header=None, index_col=None, names=column_names)

    label_columns = ['income_50k', 'marital_stat']
    categorical_columns = ['class_worker', 'det_ind_code', 'det_occ_code', 'education', 'hs_college', 'major_ind_code',
                           'major_occ_code', 'race', 'hisp_origin', 'sex', 'union_member', 'unemp_reason',
                           'full_or_part_emp', 'tax_filer_stat', 'region_prev_res', 'state_prev_res', 'det_hh_fam_stat',
                           'det_hh_summ', 'mig_chg_msa', 'mig_chg_reg', 'mig_move_reg', 'mig_same', 'mig_prev_sunbelt',
                           'fam_under_18', 'country_father', 'country_mother', 'country_self', 'citizenship',
                           'vet_question']
    train_transformed = pd.get_dummies(train_df.drop(label_columns, axis=1), columns=categorical_columns)
    test_transformed = pd.get_dummies(test_df.drop(label_columns, axis=1), columns=categorical_columns)
    train_labels = train_df[label_columns]
    test_labels = test_df[label_columns]

    # 确保测试集和训练集有相同的特征列
    missing_cols = set(train_transformed.columns) - set(test_transformed.columns)
    for c in missing_cols:
        test_transformed[c] = 0
    # 确保列顺序一致
    test_transformed = test_transformed[train_transformed.columns]

    train_income = to_categorical((train_labels.income_50k == ' 50000+.').astype(int), num_classes=2)
    train_marital = to_categorical((train_labels.marital_stat == ' Never married').astype(int), num_classes=2)
    other_income = to_categorical((test_labels.income_50k == ' 50000+.').astype(int), num_classes=2)
    other_marital = to_categorical((test_labels.marital_stat == ' Never married').astype(int), num_classes=2)

    dict_outputs = {'income': train_income.shape[1], 'marital': train_marital.shape[1]}
    dict_train_labels = {'income': train_income, 'marital': train_marital}
    dict_other_labels = {'income': other_income, 'marital': other_marital}

    validation_indices = test_transformed.sample(frac=0.5, replace=False, random_state=seed).index
    test_indices = list(set(test_transformed.index) - set(validation_indices))
    validation_data = test_transformed.iloc[validation_indices]
    validation_label = [dict_other_labels[key][validation_indices] for key in sorted(dict_other_labels.keys())]
    test_data = test_transformed.iloc[test_indices]
    test_label = [dict_other_labels[key][test_indices] for key in sorted(dict_other_labels.keys())]
    train_data = train_transformed
    train_label = [dict_train_labels[key] for key in sorted(dict_train_labels.keys())]

    return train_data, train_label, validation_data, validation_label, test_data, test_label, dict_outputs

def build_mmoe(input_dim, num_experts=6, experts_out=16, experts_hidden=32, towers_hidden=8, num_tasks=2):
    inputs = Input(shape=(input_dim,))

    # Experts
    experts = []
    for i in range(num_experts):
        x = Dense(experts_hidden, activation='relu')(inputs)
        x = Dropout(0.3)(x)
        x = Dense(experts_out)(x)
        experts.append(x)

    # 堆叠 expert 输出: (batch, num_experts, experts_out)
    expert_stack = Lambda(lambda x: K.stack(x, axis=1))(experts)

    outputs = []
    # 为每个任务定义明确的名称
    task_names = ['income_output', 'marital_output']
    for t in range(num_tasks):
        # gate (batch, num_experts)
        gate = Dense(num_experts, activation='softmax')(inputs)
        gate = Lambda(lambda g: K.expand_dims(g, axis=-1))(gate)  # (batch, num_experts, 1)

        # 加权求和 experts
        tower_input = Lambda(lambda z: K.sum(z[0] * z[1], axis=1))([expert_stack, gate])

        # Tower
        tower = Dense(towers_hidden, activation='relu')(tower_input)
        tower = Dropout(0.4)(tower)
        # 使用明确的任务名称作为输出层名称
        tower_output = Dense(1, activation='sigmoid', name=task_names[t])(tower)
        outputs.append(tower_output)

    model = Model(inputs=inputs, outputs=outputs)
    return model, task_names


# 数据准备
train_data, train_label, val_data, val_label, test_data, test_label, output_info = data_preparation()
input_dim = train_data.shape[1]

# 建模 - 获取模型和任务名称
model, task_names = build_mmoe(input_dim=input_dim, num_experts = 8, experts_out=16,
                   experts_hidden = 16, towers_hidden = 8, num_tasks = 2)

# 编译模型
model.compile(
    optimizer=tf.train.AdamOptimizer(1e-4),
    loss = 'binary_crossentropy',
    metrics = [auc_roc]
)

print(model.summary())

# 训练 - 使用与模型输出层匹配的标签名称
history = model.fit(
    train_data.values,
    {task_names[0]: train_label[0][:,1], task_names[1]: train_label[1][:,1]},
    validation_data = (
        val_data.values, 
        {task_names[0]: val_label[0][:,1], task_names[1]: val_label[1][:,1]}
    ),
    epochs = 50,
    batch_size = batch_size
)

# 测试
res = model.evaluate(
    test_data.values, 
    {task_names[0]: test_label[0][:,1], task_names[1]: test_label[1][:,1]}
)
print("Test results:", res)

# 输出每个任务的AUC
print(f"Test {task_names[0]} Loss: {res[0]}")
print(f"Test {task_names[0]} AUC: {res[1]}")
print(f"Test {task_names[1]} AUC: {res[2]}")


Using TensorFlow backend.







Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
__________________________________________________________________________________________________
Layer (type



Train on 199523 samples, validate on 49881 samples
Epoch 1/50




2025-08-20 09:21:41.267579: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2025-08-20 09:21:41.275976: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2687995000 Hz
2025-08-20 09:21:41.280616: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x169e2a80 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2025-08-20 09:21:41.280654: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2025-08-20 09:21:41.285667: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1


2025-08-20 09:21:41.568729: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-08-20 09:21:41.568893: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x1cb690d0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-08-20 09:21:41.568912: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Laptop GPU, Compute Capability 8.6
2025-08-20 09:21:41.569146: E tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:969] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-08-20 09:21:41.569189: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1618] Found device 0 with properties: 
name: NVIDIA GeForce RTX 3060 Laptop GPU major: 8 minor: 6 memoryClockRate(GH


199523/199523 [==============================] - 6s 29us/step - loss: 6.4799 - income_output_loss: 1.1427 - marital_output_loss: 5.3373 - income_output_auc_roc: 0.5145 - marital_output_auc_roc: 0.5855 - val_loss: 2.9257 - val_income_output_loss: 0.9949 - val_marital_output_loss: 1.9308 - val_income_output_auc_roc: 0.4981 - val_marital_output_auc_roc: 0.8327
Epoch 2/50
199523/199523 [==============================] - 5s 24us/step - loss: 3.6055 - income_output_loss: 1.0512 - marital_output_loss: 2.5543 - income_output_auc_roc: 0.5096 - marital_output_auc_roc: 0.7631 - val_loss: 2.4732 - val_income_output_loss: 0.9943 - val_marital_output_loss: 1.4789 - val_income_output_auc_roc: 0.4968 - val_marital_output_auc_roc: 0.8601
Epoch 3/50
199523/199523 [==============================] - 5s 23us/step - loss: 2.6937 - income_output_loss: 0.9866 - marital_output_loss: 1.7071 - income_output_auc_roc: 0.5093 - marital_output_auc_roc: 0.8035 - val_loss: 2.3576 - val_income_output_loss: 0.9935 - va

199523/199523 [==============================] - 5s 24us/step - loss: 0.6216 - income_output_loss: 0.2622 - marital_output_loss: 0.3593 - income_output_auc_roc: 0.7605 - marital_output_auc_roc: 0.9192 - val_loss: 0.4567 - val_income_output_loss: 0.2051 - val_marital_output_loss: 0.2516 - val_income_output_auc_roc: 0.8480 - val_marital_output_auc_roc: 0.9655
Epoch 23/50
199523/199523 [==============================] - 5s 24us/step - loss: 0.6014 - income_output_loss: 0.2549 - marital_output_loss: 0.3464 - income_output_auc_roc: 0.7694 - marital_output_auc_roc: 0.9236 - val_loss: 0.4456 - val_income_output_loss: 0.1995 - val_marital_output_loss: 0.2462 - val_income_output_auc_roc: 0.8521 - val_marital_output_auc_roc: 0.9673
Epoch 24/50
199523/199523 [==============================] - 5s 24us/step - loss: 0.5848 - income_output_loss: 0.2470 - marital_output_loss: 0.3378 - income_output_auc_roc: 0.7753 - marital_output_auc_roc: 0.9273 - val_loss: 0.4371 - val_income_output_loss: 0.1957 - v

199523/199523 [==============================] - 4s 21us/step - loss: 0.3795 - income_output_loss: 0.1764 - marital_output_loss: 0.2031 - income_output_auc_roc: 0.8742 - marital_output_auc_roc: 0.9730 - val_loss: 0.3118 - val_income_output_loss: 0.1565 - val_marital_output_loss: 0.1553 - val_income_output_auc_roc: 0.9053 - val_marital_output_auc_roc: 0.9852
Epoch 45/50
199523/199523 [==============================] - 5s 23us/step - loss: 0.3767 - income_output_loss: 0.1757 - marital_output_loss: 0.2010 - income_output_auc_roc: 0.8760 - marital_output_auc_roc: 0.9739 - val_loss: 0.3128 - val_income_output_loss: 0.1549 - val_marital_output_loss: 0.1579 - val_income_output_auc_roc: 0.9078 - val_marital_output_auc_roc: 0.9846
Epoch 46/50
199523/199523 [==============================] - 5s 24us/step - loss: 0.3695 - income_output_loss: 0.1745 - marital_output_loss: 0.1951 - income_output_auc_roc: 0.8764 - marital_output_auc_roc: 0.9750 - val_loss: 0.3054 - val_income_output_loss: 0.1540 - v